In [7]:
from PIL import Image
import numpy as np

# Load the image
image_path = 'preprocessed_output_2014/matched_buffer_2014.png'
image = Image.open(image_path)

# Convert image to grayscale
gray_image = image.convert('L')

# Convert the grayscale image to a numpy array
image_array = np.array(gray_image)

# Flatten the array to make it one-dimensional for easier sorting
flat_array = image_array.flatten()

# Find the indices of the top 5 lightest pixels
# Since we are looking for the lightest, we sort in descending order
indices = np.argsort(flat_array)[-5:]

# Convert 1D indices back to 2D coordinates
rows, cols = image_array.shape
top_pixels = [(index // cols, index % cols) for index in indices]

# Print the coordinates and their corresponding pixel values
for coord in top_pixels:
    print(f"Coordinates: {coord}, Pixel Value: {image_array[coord]}")

Coordinates: (np.int64(62), np.int64(100)), Pixel Value: 248
Coordinates: (np.int64(61), np.int64(81)), Pixel Value: 249
Coordinates: (np.int64(62), np.int64(102)), Pixel Value: 250
Coordinates: (np.int64(61), np.int64(80)), Pixel Value: 251
Coordinates: (np.int64(62), np.int64(101)), Pixel Value: 254


In [6]:
from PIL import Image
import numpy as np

# Load the .tif image
image_path = 'preprocessed_output_2014/matched_buffer_2014.tif'  # Update the path if necessary
image = Image.open(image_path)

# Convert image to grayscale to find the lightest pixels
gray_image = image.convert('L')
image_array = np.array(gray_image)

# Flatten the array to make it one-dimensional for easier sorting
flat_array = image_array.flatten()

# Find the indices of the top 5 lightest pixels
indices = np.argsort(flat_array)[-5:]

# Convert 1D indices back to 2D coordinates
rows, cols = image_array.shape
top_pixels = [(index // cols, index % cols) for index in indices]

# Load the original image again to mark the pixels in color
color_image = image.convert('RGB')
pixels = color_image.load()

# Mark the pixels with red color
for coord in top_pixels:
    pixels[coord[1], coord[0]] = (255, 0, 0)  # Set to red

# Display the modified image
color_image.show()

# Optionally, save the modified image
color_image.save('preprocessed_output_2014/marked_matched_buffer_2014.tif')

UnidentifiedImageError: cannot identify image file 'preprocessed_output_2014/matched_buffer_2014.tif'

In [8]:
from PIL import Image
import numpy as np

# Ensure Pillow is up to date and has the necessary support for TIFF files
# You might need to restart your Jupyter kernel after installing dependencies

# Load the .tif image
image_path = '/Users/devparagiri/Research/tree-mortality-v3/preprocessed_output_2014/matched_buffer_2014.tif'  # Full path
try:
    image = Image.open(image_path)
except IOError:
    print("Error: The file could not be opened. Check the file path and file integrity.")

# Convert image to grayscale to find the lightest pixels
gray_image = image.convert('L')
image_array = np.array(gray_image)

# Flatten the array to make it one-dimensional for easier sorting
flat_array = image_array.flatten()

# Find the indices of the top 5 lightest pixels
indices = np.argsort(flat_array)[-5:]

# Convert 1D indices back to 2D coordinates
rows, cols = image_array.shape
top_pixels = [(index // cols, index % cols) for index in indices]

# Load the original image again to mark the pixels in color
color_image = image.convert('RGB')
pixels = color_image.load()

# Mark the pixels with red color
for coord in top_pixels:
    pixels[coord[1], coord[0]] = (255, 0, 0)  # Set to red

# Display the modified image
color_image.show()

# Save the modified image to a new file to avoid overwriting the original
output_path = '/Users/devparagiri/Research/tree-mortality-v3/preprocessed_output_2014/marked_matched_buffer_2014.png'
color_image.save(output_path)

Error: The file could not be opened. Check the file path and file integrity.
